In [3]:
using Oceananigans
using JLD2
using CairoMakie
using Statistics
using ImageFiltering: imfilter, Kernel.gaussian
using OffsetArrays: no_offset_view
using CairoMakie.Colors

In [4]:
runnames = ["Ri000", "Ri010", "Ri020", "Ri030", "Ri050"]
foldernames = map(runnames) do runname
    "../../../scratch/filament-instability/$runname"
end
runlabels = [L"0.00", L"0.10", L"0.20", L"0.30", L"0.50"]
legendtitle = L"Ri";
b_levels = range(-300, -227, 12)
ψ_levels = range(-0.009, 0.009, 12)
ψ_colors = [
    to_colormap(:PuOr)[end], 
    to_colormap(:PuOr)[end], 
    to_colormap(:PuOr)[end], 
    RGB(1, 1, 1), 
    to_colormap(:PuOr)[1], 
    to_colormap(:PuOr)[1],
    to_colormap(:PuOr)[1]
]
VSP_colors = reverse(to_colormap(:curl))

# Vertical velocity slices and VSP

In [ ]:
map(foldernames[1:3], ["00", "01", "02"]) do foldername, Ri 
#foldername = foldernames[5]
z = -0.05

init_time, run_time, sp = jldopen("$foldername/parameters.jld2") do file
    file["parameters/init_time"], file["parameters/run_time"], file["parameters/simulation"]
end

xsᶜ, ysᶜ, zsᶜ, zsᶠ, iterations, ts = jldopen("$foldername/TKE.jld2") do file

    xsᶜ = no_offset_view(file["grid/xᶜᵃᵃ"])
    zsᶜ = no_offset_view(file["grid/zᵃᵃᶜ"])
    ysᶜ = no_offset_view(file["grid/yᵃᶜᵃ"])
    zsᶠ = no_offset_view(file["grid/zᵃᵃᶠ"])
    
    iterations = keys(file["timeseries/t"])
    ts = map(iteration->(file["timeseries/t/$iteration"] - init_time)/(2π), iterations)
    
    xsᶜ, ysᶜ, zsᶜ, zsᶠ, iterations, ts
end

slice = argmin(abs.(zsᶠ .- z))

n = Observable(1)

iteration = @lift iterations[$n]
t = @lift ts[$n]

VSP = @lift jldopen(file->no_offset_view(file["timeseries/VSP/$($iteration)"][:, 1, :]), "$foldername/TKE.jld2")
b_dfm = @lift jldopen(file->no_offset_view(file["timeseries/b_dfm/$($iteration)"][:, 1, :]), "$foldername/DFM.jld2")

w = @lift jldopen(file->no_offset_view(file["timeseries/w/$($iteration)"][:, 512, :]), "$foldername/output.jld2")
b = @lift jldopen(file->no_offset_view(file["timeseries/b/$($iteration)"][:, 512, :]), "$foldername/output.jld2")

w_slice =  @lift imfilter(transpose(jldopen(file->no_offset_view(file["timeseries/w/$($iteration)"])[:, :, slice], "$foldername/output.jld2")), gaussian((1, 1)))
b_slice =  @lift transpose(jldopen(file->no_offset_view(file["timeseries/b/$($iteration)"])[:, :, slice], "$foldername/output.jld2"))

title = @lift "Riₘᵢₙ=$(round(sp.Ri, digits=2)), t/2π=$(round($t, digits=2))"

fig = Figure(; size=(1000, 600)./0.8, fontsize=14)
axis_kwargs = (; xlabel=L"x", ylabel=L"z", limits=(-2.5sp.L, -0sp.L, -1.2sp.H, 0), xlabelsize=16, ylabelsize=16)

ax_w = Axis(fig[1, 1]; axis_kwargs...)
ax_VSP = Axis(fig[1, 3]; axis_kwargs...)
ax_w_slice = Axis(fig[2, 1:3]; xlabel=L"y", ylabel=L"x", limits=(-5sp.L, 5sp.L, -2sp.L, 0sp.L), xlabelsize=16, ylabelsize=16)

w_max = 0.2
VSP_max = 0.08
ht_w = heatmap!(ax_w, xsᶜ, zsᶠ, w; colormap=:balance, colorrange=(-w_max, w_max))
ht_VSP = heatmap!(ax_VSP, xsᶜ, zsᶜ, VSP; colormap=VSP_colors, colorrange=(-VSP_max, VSP_max))
contour!(ax_VSP, xsᶜ, zsᶜ, b_dfm; color=(:black, 0.5), levels=b_levels, linewidth=2)
contour!(ax_w, xsᶜ, zsᶜ, b_dfm; color=(:black, 0.5), levels=b_levels, linewidth=2)
lines!(ax_w, [xsᶜ[1], xsᶜ[end]], [z, z]; color=(:red, 0.3), linestyle=:dash, linewidth=2)

heatmap!(ax_w_slice, ysᶜ, xsᶜ, w_slice; colormap=:balance, colorrange=(-w_max, w_max))
contour!(ax_w_slice, ysᶜ, xsᶜ, b_slice; color=(:black, 0.5), levels=b_levels, linewidth=2)

Colorbar(fig[1, 2], ht_w; label=L"w")
Colorbar(fig[2, 4], ht_w; label=L"w")
Colorbar(fig[1, 4], ht_VSP; label="VSP")
hideydecorations!.(ax_VSP)
colgap!(fig.layout, 1, 30)
#colgap!(fig.layout, 2, 30)
Label(fig[0, :], title)
gl = GridLayout(fig[1, 1], 
        tellwidth = false, 
        tellheight = false, 
        halign = :left, 
        valign = :top,
    )
    Box(gl[1, 1], color = :white, strokecolor = :black, strokewidth = 0)
    Label(gl[1, 1], "a)",
        fontsize = 14,
        padding = (1, 1, 2, 2),
    )
    gl = GridLayout(fig[1, 3], 
        tellwidth = false, 
        tellheight = false, 
        halign = :left, 
        valign = :top,
    )
    Box(gl[1, 1], color = :white, strokecolor = :black, strokewidth = 0)
    Label(gl[1, 1], "b)",
        fontsize = 14,
        padding = (1, 1, 2, 2),
    )
    
    gl = GridLayout(fig[2, 1], 
        tellwidth = false, 
        tellheight = false, 
        halign = :left, 
        valign = :top,
    )
    Box(gl[1, 1], color = :white, strokecolor = :black, strokewidth = 0)
    Label(gl[1, 1], "c)",
        fontsize = 16,
        padding = (1, 1, 2, 2),
    )
fig

record(fig, "../output/videos/w_slice_$Ri.mp4", 1:length(iterations); framerate=6) do i
    n[] = i
    print("$i\r")
end
end

# Filament state and potential vorticity

In [ ]:
foldername = foldernames[5]
map(foldernames[1:3], ["00", "01", "02"]) do foldername, Ri 
init_time, run_time, sp = jldopen("$foldername/parameters.jld2") do file
    file["parameters/init_time"], file["parameters/run_time"], file["parameters/simulation"]
end
xs, zs, iterations, ts = jldopen("$foldername/DFM.jld2") do file
    iterations = keys(file["timeseries/t"])
    ts = map(iteration->(file["timeseries/t/$iteration"] - init_time)/(2π), iterations)
    # All of these fields are on Center, Nothing, Center
    xs = no_offset_view(file["grid/xᶜᵃᵃ"])
    zs = no_offset_view(file["grid/zᵃᵃᶜ"])
    xs, zs, iterations, ts
end

n = Observable(1)

iteration = @lift iterations[$n]
t = @lift ts[$n]

v_dfm = @lift jldopen(file->no_offset_view(file["timeseries/v_dfm/$($iteration)"][:, 1, :]), "$foldername/DFM.jld2")
b_dfm = @lift jldopen(file->no_offset_view(file["timeseries/b_dfm/$($iteration)"][:, 1, :]), "$foldername/DFM.jld2")
q_dfm = @lift jldopen(file->no_offset_view(file["timeseries/q_dfm/$($iteration)"][:, 1, :]), "$foldername/PV.jld2")
ψ = @lift jldopen(file->no_offset_view(file["timeseries/ψ/$($iteration)"][:, 1, :]), "$foldername/DFM.jld2")

title = @lift "Riₘᵢₙ=$(round(sp.Ri, digits=2)), t/2π=$(round($t, digits=2))"

v_max=2
q_max=200

fig = Figure(; size=(1000, 600)./1.3, fontsize=14)
axis_kwargs = (; xlabel=L"x", ylabel=L"z", limits=(-3.3sp.L, 3.3sp.L, -1.2sp.H, 0), xlabelsize=16, ylabelsize=16)

ax_v = Axis(fig[2, 1]; axis_kwargs..., title=L"\overline{v}")
ax_q = Axis(fig[2, 2]; axis_kwargs..., title=L"\overline{q}")
hideydecorations!(ax_q)

ht_v = heatmap!(ax_v, xs, zs, v_dfm, colormap=:balance, colorrange=(-v_max, v_max))
contour!(ax_v, xs, zs, b_dfm; color=(:black, 0.5), levels=b_levels, linewidth=2)
contour!(ax_v, xs, zs, ψ; colormap=ψ_colors, linestyle=:dash, levels=ψ_levels, linewidth=1.5)

ht_q = heatmap!(ax_q, xs, zs, q_dfm, colormap=:balance, colorrange=(-q_max, q_max), highclip=to_colormap(:balance)[end])
contour!(ax_q, xs, zs, b_dfm; color=(:black, 0.5), levels=b_levels, linewidth=2)
#contour!(ax_q, xs, zs, ψ; colormap=ψ_colors, linestyle=:dash, levels=ψ_levels, linewidth=1.5)

Colorbar(fig[3, 1], ht_v; vertical=false)
Colorbar(fig[3, 2], ht_q; vertical=false)

Label(fig[1, :], title)
rowgap!(fig.layout, 1, 0)
colgap!(fig.layout, 30)

gl = GridLayout(fig[2, 1], 
        tellwidth = false, 
        tellheight = false, 
        halign = :left, 
        valign = :top,
    )
    Box(gl[1, 1], color = :white, strokecolor = :black, strokewidth = 0)
    Label(gl[1, 1], "a)",
        fontsize = 14,
        padding = (1, 1, 2, 2),
    )
gl = GridLayout(fig[2, 2], 
        tellwidth = false, 
        tellheight = false, 
        halign = :left, 
        valign = :top,
    )
    Box(gl[1, 1], color = :white, strokecolor = :black, strokewidth = 0)
    Label(gl[1, 1], "b)",
        fontsize = 14,
        padding = (1, 1, 2, 2),
    )

fig

record(fig, "../output/videos/v_$Ri.mp4", 1:length(iterations); framerate=6) do i
    n[] = i
    print("$i\r")
end
end